### IMPORTAÇÕES 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sympy as symbols
from pandas.api.types import is_numeric_dtype

### DATASET 

In [8]:
input_file = 'C:\\JupyterLab\\PAPL-UFPE\\ML-Fuzzy\\Testes\\iris.data'
output_file = 'iris.csv'
iris_data = pd.read_csv(input_file, header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
iris_data.to_csv(output_file, index=False)

# normalização 
for index in iris_data.columns: 
    if is_numeric_dtype(iris_data[index][0]):
        iris_data[index] = iris_data[index]/max(iris_data[index])
        
iris_data = np.array(iris_data)
iris_data

array([[0.6455696202531644, 0.7954545454545454, 0.20289855072463767,
        0.08, 'Iris-setosa'],
       [0.620253164556962, 0.6818181818181818, 0.20289855072463767, 0.08,
        'Iris-setosa'],
       [0.5949367088607594, 0.7272727272727273, 0.18840579710144928,
        0.08, 'Iris-setosa'],
       [0.5822784810126581, 0.7045454545454545, 0.21739130434782608,
        0.08, 'Iris-setosa'],
       [0.6329113924050632, 0.8181818181818181, 0.20289855072463767,
        0.08, 'Iris-setosa'],
       [0.6835443037974683, 0.8863636363636362, 0.24637681159420288,
        0.16, 'Iris-setosa'],
       [0.5822784810126581, 0.7727272727272726, 0.20289855072463767,
        0.12, 'Iris-setosa'],
       [0.6329113924050632, 0.7727272727272726, 0.21739130434782608,
        0.08, 'Iris-setosa'],
       [0.5569620253164557, 0.6590909090909091, 0.20289855072463767,
        0.08, 'Iris-setosa'],
       [0.620253164556962, 0.7045454545454545, 0.21739130434782608, 0.04,
        'Iris-setosa'],
       [0.68

In [9]:
# tirando a parte textual de classificação da espécie
iris_data_without_last_column = iris_data[:, :-1]
iris_data_without_last_column

array([[0.6455696202531644, 0.7954545454545454, 0.20289855072463767,
        0.08],
       [0.620253164556962, 0.6818181818181818, 0.20289855072463767, 0.08],
       [0.5949367088607594, 0.7272727272727273, 0.18840579710144928,
        0.08],
       [0.5822784810126581, 0.7045454545454545, 0.21739130434782608,
        0.08],
       [0.6329113924050632, 0.8181818181818181, 0.20289855072463767,
        0.08],
       [0.6835443037974683, 0.8863636363636362, 0.24637681159420288,
        0.16],
       [0.5822784810126581, 0.7727272727272726, 0.20289855072463767,
        0.12],
       [0.6329113924050632, 0.7727272727272726, 0.21739130434782608,
        0.08],
       [0.5569620253164557, 0.6590909090909091, 0.20289855072463767,
        0.08],
       [0.620253164556962, 0.7045454545454545, 0.21739130434782608, 0.04],
       [0.6835443037974683, 0.8409090909090908, 0.21739130434782608,
        0.08],
       [0.6075949367088607, 0.7727272727272726, 0.2318840579710145, 0.08],
       [0.607594936

### CÓDIGO

> Aqui ele está mais objetivo, com mais comentários e alterações consulte os documentos de teste

In [10]:
# valor definido pois conheço o dataset
K = 3

# fuzziness value 
m = [1.1, 1.6, 2]
# valores da lista indicados pelo professor

# membership
U = np.random.rand(iris_data_without_last_column.shape[0], K)
print('Membership inicial (U inicial):')
print(U)
print()


def calculando_centroides(iris_data_without_last_column, K, U, m):
    centroides = np.zeros((K, iris_data_without_last_column.shape[1]))
    for i in range(K):
        U_corrigido = np.tile(U[:, i][:, np.newaxis], (1, iris_data_without_last_column.shape[1]))
        m_expandido = np.tile(m[i], (iris_data_without_last_column.shape[0], iris_data_without_last_column.shape[1]))
        centroides[i, :] = np.sum((U_corrigido ** m_expandido) * iris_data_without_last_column, axis=0) / np.sum(U_corrigido ** m_expandido)
    return centroides

centroides = calculando_centroides(iris_data_without_last_column, K, U, m)

def matriz_dg(iris_data_without_last_column, centroides):
    num_points = iris_data_without_last_column.shape[0]
    num_centroides = centroides.shape[0]

    distancias = np.zeros((num_points, num_centroides))

    for i in range(num_points):
        for j in range(num_centroides):
            distancias[i, j] = np.linalg.norm(iris_data_without_last_column[i] - centroides[j])

    return distancias

d_matriz = matriz_dg(iris_data_without_last_column, centroides)

# Exibe a matriz de distâncias
print("Matriz de Distâncias:")
print(d_matriz)

def fun_objetivo(U, matriz_dg, m):
    c, n = U.shape
    objetivo = 0

    for i in range(c):
        for j in range(n):
            objetivo += U[i, j] ** m * matriz_dg[i, j] ** 2

    return objetivo

def membership_c(iris_data_without_last_column, centroides, K, m):
    U_novo = np.zeros_like(d_matriz)

    for i in range(K):
        U_novo[:, i] = d_matriz[:, i]

    for j in range(len(m)):
        U_novo = 1/(U_novo ** (2/(m[j]-1)) * np.sum((1/U_novo) ** (2/(m[j]-1)), axis=1)[:, np.newaxis])

    return U_novo

U_novo = membership_c(iris_data_without_last_column, centroides, K, m)
N = iris_data_without_last_column.shape[0] # número de dados
velho_obj = fun_objetivo(U_novo, d_matriz, m)

maximo = 100

# Inicializar valores de pertinência e centroides
U = np.random.rand(iris_data_without_last_column.shape[0], K)
centroides = calculando_centroides(iris_data_without_last_column, K, U, m)

# Calcular matriz de distâncias inicial
J_anterior = matriz_dg(iris_data_without_last_column, centroides)

for iteration in range(maximo):
    # atualizar valores 
    U_novo = membership_c(iris_data_without_last_column, centroides, K, m)
    centroides = calculando_centroides(iris_data_without_last_column, K, U_novo, m)
    
    nova_J = matriz_dg(iris_data_without_last_column, centroides)

    novo_objetivo = fun_objetivo(U_novo, nova_J, m)

    # verificando a convergência
    if (np.all(nova_J <= J_anterior) or 
        np.all(np.abs(novo_objetivo - velho_obj) < 1e-6) or
        iteration == maximo - 1):
        break

Membership inicial (U inicial):
[[0.68279098 0.88964211 0.27975223]
 [0.08403518 0.54296114 0.59260239]
 [0.07582988 0.70851361 0.24416615]
 [0.63753634 0.23437067 0.3761066 ]
 [0.68716166 0.21052248 0.19602053]
 [0.14232037 0.65392812 0.01132742]
 [0.60515133 0.5720724  0.44716827]
 [0.97841104 0.90115175 0.12042495]
 [0.8994982  0.77040795 0.83863418]
 [0.97799431 0.79421805 0.92732048]
 [0.49996363 0.51843915 0.38219352]
 [0.31827169 0.52333623 0.33230903]
 [0.71842933 0.59373008 0.63643309]
 [0.90462772 0.86900532 0.66796634]
 [0.46432831 0.85275882 0.21922566]
 [0.4485645  0.17634644 0.00904192]
 [0.74892735 0.42195859 0.58793315]
 [0.40827471 0.05602168 0.10098001]
 [0.26025086 0.5039039  0.54164191]
 [0.88428354 0.20386782 0.24689264]
 [0.0556997  0.32277199 0.49141969]
 [0.76683662 0.27103743 0.1284063 ]
 [0.27136455 0.96007962 0.54437992]
 [0.58828187 0.344929   0.6711459 ]
 [0.26797733 0.25470504 0.55532959]
 [0.12163758 0.46631822 0.12525636]
 [0.93320179 0.39985761 0.940497

In [11]:
# atualizações
velho_obj = novo_objetivo
J_anterior = nova_J
U = U_novo

print("Centroides finais:")
print(centroides)
print()
print("Membership final (U_novo):")
print(U)

Centroides finais:
[[0.18450387 0.17382081 0.13509618 0.11874118]
 [0.18092466 0.17883127 0.1216661  0.10353263]
 [0.18812713 0.16919358 0.14764749 0.13251639]]

Membership final (U_novo):
[[0.36331305 0.30379824 0.33288872]
 [0.36351003 0.29040808 0.34608189]
 [0.36665944 0.3030963  0.33024426]
 [0.36766326 0.29322971 0.33910704]
 [0.36428954 0.30718517 0.32852529]
 [0.3657101  0.29236798 0.34192192]
 [0.37142844 0.29846665 0.3301049 ]
 [0.36410428 0.29896001 0.33693571]
 [0.3695565  0.29206887 0.33837463]
 [0.36043015 0.29906529 0.34050456]
 [0.36118428 0.30334266 0.33547306]
 [0.36608392 0.29763098 0.3362851 ]
 [0.36075305 0.30089055 0.3383564 ]
 [0.36566488 0.32229555 0.31203957]
 [0.35848411 0.31333753 0.32817836]
 [0.36291626 0.30559232 0.33149142]
 [0.36569    0.30157421 0.33273579]
 [0.36616937 0.29693746 0.33689317]
 [0.36130512 0.29354696 0.34514792]
 [0.36609875 0.30199841 0.33190283]
 [0.36031989 0.29017199 0.34950812]
 [0.36860371 0.29349315 0.33790314]
 [0.36687185 0.3244